# Challenge

Let's add a new feature:

- **Day of Week (Categorical)**

Consider these questions:

- How would you calculate this feature?
- How would you represent it in the data?
- Where would you store this feature?
- How would this feature affect your pipeline? What would you need to rework?